In [ ]:
using DrWatson
@quickactivate "masterarbeit"

In [ ]:
using Random
using LaTeXStrings
using GLMakie
using StatsBase

In [ ]:
Makie.inline!(true)
fontsize_theme = Theme(fontsize=35)
set_theme!(fontsize_theme)
wblue = Makie.wong_colors()[1]
worange = Makie.wong_colors()[2]
wgreen = Makie.wong_colors()[3]
wpink = Makie.wong_colors()[4]
wlblue = Makie.wong_colors()[5]
worange = Makie.wong_colors()[6]
wyellow = Makie.wong_colors()[7];

In [ ]:
using Revise

In [ ]:
using masterarbeit

In [ ]:
include(joinpath(projectdir(),"src/vegas.jl"))

In [ ]:
omega = 5.12
dphi = 10.0

In [ ]:
ytozmap = trident_phasespace(omega)

In [ ]:
ytozmap.Eb_lim

In [ ]:
f = x -> dσpT(omega, x...)
vegasstart = [1.0, -1.0, 0.0, 1.0, 0.89]
vegasend = [1.0+ytozmap.Ea_lim, 1.0, 2.0*pi, 1.0+ytozmap.Eb_lim, 1.0]

In [ ]:
res = vegas(f, vegasstart, vegasend)
weights = res.weights;

In [ ]:
function vegaswithgrid()
    fig = Figure(size=(1000,1000), figure_padding=40)
    ax2 = Axis(fig[1,1], xticks=xticks, yticks=yticks,
        aspect=1, xlabel=L"\cos{\theta}", ylabel=L"\overline{\omega'}")
    zs = [f([x,y])[1] for x in xs, y in ys]
    hm = heatmap!(xs, ys, zs)
    vlines!(res.adaptive_grid[:,1][1:50:end], color=:black, linewidth=1)
    hlines!(res.adaptive_grid[:,2][1:50:end], color=:black, linewidth=1)
    save("compton_vegas_grid.png", fig)
    return fig
end 
fig = vegaswithgrid()
fig

In [ ]:
samples, Js = sample_from_adaptive_grid(res, 10^5)


In [ ]:
sx = [s[1] for s in samples]
sy = [s[4] for s in samples]
histo = fit(Histogram, (sx, sy), nbins=100)
histo_n = StatsBase.normalize(histo, mode=:pdf)
fig = Figure(size=(1000,1000), figure_padding=40)
ax = Axis(fig[1,1], xlabel=L"E_a", ylabel=L"E_b", aspect=1)
hm = heatmap!(histo.edges[1], histo.edges[2], histo_n.weights)
fig[1, 2] = GridLayout(width = 20)
Colorbar(fig[1,3], hm, width=40)
save("trident_vegas_samples.png", fig)
fig

In [ ]:
fevals = sample -> dσpT(omega, sample[1], sample[2], sample[3], sample[4], sample[5]);
weights = Js .* fevals.(samples);

In [ ]:
wi = weights
w_avg = mean(wi)
w_max = maximum(wi)
wi_n = wi ./ w_max
w_avg_n = mean(wi_n)
println("mean weight = $(mean(wi))")
println("max weight = $(maximum(wi))")
println("unweighting efficiency = $(mean(wi)/maximum(wi))")

In [ ]:
using JLD2

In [ ]:
#save_object("vegas_weights.jld2", wi)